In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import os
import sys
from itertools import chain

import folium

import src.mex as mex
import src.mex.regions2010 as region
import src.utils.map_vis as mvis
import src.mex.tower as tower
import src.tower_interpolation as tw_int
import src.mex_helper as mex_helper
if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')


# load geoms

In [4]:

zms = region.mpa_all(to_4326=True)
mun_ids = list(chain(*zms.mun_ids.apply(lambda x: x.split(',')).tolist()))
mgm_zms = region.municipalities(mun_ids, to_4326=True)
mglu_zms = region.mpa_urban_per_municipality(to_4326=True)
mglr_zms = region.locs_rural(mun_ids, to_4326=True)


In [5]:
zms = region.mpa_all(to_4326=True)
zms = zms.iloc[:10]
mun_ids = list(chain(*zms.mun_ids.apply(lambda x: x.split(',')).tolist()))

zms_agebs = region.agebs(mun_ids,to_4326=True)

tvor = tower.voronoi(to_4326=True)
tvor_x_zms = tower.voronoi_x_region('mpa')
tvor_x_zms = tvor_x_zms[tvor_x_zms.CVE_SUN.isin(zms.index.astype(str))]
zms_tvors= tvor.loc[set(tvor_x_zms.gtid)]

loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson


In [6]:
t_pts = tower.pts(to_4326=True)
t_pts.geometry = t_pts.buffer(0.001, resolution=2)
t_pts.set_index('gtid',inplace=True)
zms_t_pts = t_pts.loc[zms_tvors.index]

# mpa to localidads

In [ ]:

print('building map')
m = folium.Map(location=[mex.clat, mex.clon], zoom_start=5)
mvis.geojson_per_row(zms, name='metro', tip_cols=['NOM_SUN', 'CVE_SUN', 'pobtot'], color='grey', some_map=m)
mvis.geojson_per_row(mgm_zms, name='mgm', tip_cols=['NOM_MUN', 'mun_id'], color='yellow', some_map=m)
mvis.geojson_per_row(mglu_zms, name='mglu_zms', tip_cols=['CVE_SUN', 'NOM_LOC', 'mun_id', 'loc_id', 'pobtot'],
                     color='green', some_map=m)
mvis.geojson_per_row(mglr_zms, name='mglr_zms', tip_cols=['NOM_LOC', 'mun_id', 'loc_id', 'pobtot'], color='green',
                     some_map=m)

folium.LayerControl().add_to(m)
m.save('maps/Mex zms to localidads.html')


# idw vs grid_by_area

In [14]:
import pandas as pd

In [15]:
import src.utils.idw as idw

In [16]:
call_direction = 'out+in'
aver = mex_helper.stat_tw_dow_aver_hr_uniq_user(call_direction)
tw_avg = pd.DataFrame(aver['wd']).T

loading cached tw average stats/stat_tw_dow_aver_hr_uniq_user-out+in.pickle


In [61]:
zms_t_pts['t12'] = tw_avg['12']

zms_t_pts['t12'].fillna(0,inplace=True)

In [19]:
t2g2000_area = tw_int.to_mpa_grids(2000, by='area', per_mun=False)
t2g2000_area.set_index('grid', inplace=True)
avg_g2000_area = tw_int.interpolate_stats(tw_avg, t2g2000_area)

to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g2000_by_area.csv


In [57]:
avg_idw2000 = tw_int.interpolate_idw(tw_avg, 2000, per_mun=False)

interpolate_idw loading existing file data/mex_tw_intpl/interpolate_idw10_g2000.csv


In [21]:
zms_g2000 = region.mpa_grids(2000,per_mun=False)

reading existing grids


In [ ]:

tws_mex = mex.tower.pts()
zms_mex = region.mpa_all()
tws_x_zms = gp.sjoin(tws_mex, zms_mex)[['gtid', 'index_right']]

In [17]:
import geopandas as gp
import numpy as np
print('computing weight for each SUN')
gs_avg = []
for sun in zms_g2000.CVE_SUN.unique():
    
    zm_grids = zms_g2000[zms_g2000.CVE_SUN == sun]
    zm_tws = tws_mex[tws_mex.gtid.isin(tws_x_zms[tws_x_zms.index_right == sun].gtid)]
    zm_g_coords = np.array(zm_grids.geometry.apply(lambda x: x.centroid.coords[0]).tolist())
    zm_t_coords = np.array(zm_tws.geometry.apply(lambda x: x.coords[0]).tolist())
    idw_tree = idw.tree(zm_t_coords, tw_avg.reindex(zm_tws.gtid, fill_value=0)['12'])
    k = 10 if len(zm_t_coords)>10 else len(zm_t_coords)
    g_avg = pd.DataFrame(idw_tree(zm_g_coords,k=k), index=zm_grids.index)
    gs_avg.append(g_avg)
    break



computing weight for each SUN


In [58]:
zms_g2000['g_area'] = avg_g2000_area['12']
zms_g2000['idw_k10'] = avg_idw2000['12']


In [59]:
zm13_g2000 = zms_g2000[zms_g2000.CVE_SUN==13].to_crs(epsg=4326)
zm13_g2000['g_area_norm'] = zm13_g2000.g_area/zm13_g2000.g_area.max()
zm13_g2000['idw_k10_norm'] = zm13_g2000.idw_k10/zm13_g2000.idw_k10.max()

In [63]:

print('building map')
m = folium.Map(location=[mex.clat, mex.clon], zoom_start=5)
mvis.geojson_per_row(zms.reset_index(), name='metro', tip_cols=['NOM_SUN', 'CVE_SUN', 'pobtot'], color='grey', some_map=m)
mvis.geojson_per_row_color_col(zm13_g2000.reset_index(), name='zm13_g2000_area', tip_cols=['grid', 'CVE_SUN', 'g_area', 'idw_k10'], color_col='g_area_norm', some_map=m)
mvis.geojson_per_row_color_col(zm13_g2000.reset_index() , name='zm13_g2000_idw_k10', tip_cols=['grid', 'CVE_SUN', 'g_area', 'idw_k10'], color_col='idw_k10_norm', some_map=m)

mvis.geojson_per_row_color_col(zms_t_pts.reset_index(), name='tower', tip_cols=['gtid','lonlat','t12'], some_map=m)


folium.LayerControl().add_to(m)
m.save('maps/Mex zms grids idw.html')


building map


# tower interpolation to agebs

In [57]:
tvor_x_agebs = tower.voronoi_x_region('mga')
covered = tvor_x_agebs[tvor_x_agebs.gtid.isin(zms_tvors.index)]
covered_ageb_ids = covered.ageb_id.unique()
covered_loc_ids = sorted(set([aid[:9] for aid in covered_ageb_ids]))
covered_mun_ids = sorted(set([aid[:5] for aid in covered_ageb_ids]))
covered_agebs = region.agebs(mun_ids=covered_mun_ids, loc_ids=covered_loc_ids, to_4326=True)

In [58]:
extra_agebs = covered_agebs.loc[set(covered_agebs.index) - set(zms_agebs.index)]

In [84]:
t2ageb_area = tw_int.to_mpa_agebs('area')
t2ageb_pop = tw_int.to_mpa_agebs('pop')
zms_t_pts['weight_area']=t2ageb_area.groupby('tower').weight.sum()
zms_t_pts['weight_pop']=t2ageb_pop.groupby('tower').weight.sum()

to_map_agebs loading existing file data/mex_tw_intpl/tower_to_mpa_agebs_by_area.csv
to_map_agebs loading existing file data/mex_tw_intpl/tower_to_mpa_agebs_by_pop.csv


In [95]:

m = folium.Map(location=[mex.clat, mex.clon], zoom_start=5)
mvis.geojson_per_row(zms.reset_index(), name='metro', tip_cols=['NOM_SUN', 'CVE_SUN', 'pobtot'], color='grey', some_map=m)
mvis.geojson_per_row(zms_agebs.reset_index(), name='agebs', tip_cols=['ageb_id', 'pobtot'], color='green', some_map=m)
mvis.geojson_per_row(extra_agebs.reset_index(), name='extra_agebs', tip_cols=['ageb_id', 'pobtot'], color='red', some_map=m)
mvis.geojson_per_row(zms_tvors.reset_index(), name='tvor', tip_cols=['gtid'], color='blue', some_map=m)
mvis.geojson_per_row_color_col(zms_t_pts[~zms_t_pts.weight_pop.isnull()].reset_index(), name='tower', tip_cols=['gtid','weight_area', 'weight_pop','lonlat'], color_col='weight_pop', some_map=m)
mvis.geojson_per_row(zms_t_pts[zms_t_pts.weight_pop.isnull()].reset_index(), name='tower_no_weight', tip_cols=['gtid','weight_area', 'weight_pop','lonlat'], some_map=m)


folium.LayerControl().add_to(m)
m.save('maps/Mex tower to mpa ageb.html')

# grids and mun

In [5]:
zms_g1000 = region.mpa_grids(1000,per_mun=False)
zms_g1000_mun = region.mpa_grids(1000,per_mun=True)

reading existing grids
reading existing grids
dividing grids by municipality
computing the intersection between p1 grid and p2 mgmu
computing area of the intersections
computing the weight


In [12]:
zms_g2000 = region.mpa_grids(2000,per_mun=False,to_4326=True)
zms_g2000_mun = region.mpa_grids(2000,per_mun=True,to_4326=True)

reading existing grids
reading existing grids
dividing grids by municipality
computing the intersection between p1 grid and p2 mgmu
computing area of the intersections
computing the weight


In [14]:

print('building map')
m = folium.Map(location=[mex.clat, mex.clon], zoom_start=5)
mvis.geojson_per_row(zms.reset_index(), name='metro', tip_cols=['NOM_SUN', 'CVE_SUN', 'pobtot'], color='grey', some_map=m)
mvis.geojson_per_row(mgm_zms.reset_index(), name='mgm', tip_cols=['NOM_MUN', 'mun_id'], color='yellow', some_map=m)
mvis.geojson_per_row(zms_g2000_mun.reset_index(), name='zms_g2000_mun', tip_cols=['grid', 'CVE_SUN', 'mun_id'], color='blue', some_map=m)
mvis.geojson_per_row(zms_g2000.reset_index(), name='zms_g2000', tip_cols=['grid', 'CVE_SUN'], color='green', some_map=m)


folium.LayerControl().add_to(m)
m.save('maps/Mex zms grids per mun.html')


building map
